# 클라이언트 API호출

In [1]:
import pandas as pd
import numpy as np
import requests
import json

## 1. 단일 값 호출

In [2]:
import requests

input = {'INCOME': 107677,'OVERAGE': 155,
         'LEFTOVER': 23,
         'HOUSE': 154007.0,
         'HANDSET_PRICE': 859,
         'OVER_15MINS_CALLS_PER_MONTH': 15,
         'AVERAGE_CALL_DURATION': 4,
         'REPORTED_SATISFACTION':  'very_sat',
         'REPORTED_USAGE_LEVEL': 'little'}

response = requests.post('http://127.0.0.1:8080/predict/', data = input)
result = response.json() # 결과를 json 형태로 받아와 저장하지만, python에서는 딕셔너리로 저장됨.
result

{'pred1': [1], 'pred2': ['Leave']}

## 2. 여러 값 호출

In [3]:
import requests

input = {'INCOME': [107677, 64070],
         'OVERAGE': [155, 228],
         'LEFTOVER': [23, 0],
         'HOUSE': [154007.0, 326646.0],
         'HANDSET_PRICE': [859, 275],
         'OVER_15MINS_CALLS_PER_MONTH': [15, 28],
         'AVERAGE_CALL_DURATION': [4, 9],        
         'REPORTED_SATISFACTION':  ['very_sat', 'sat'],    
         'REPORTED_USAGE_LEVEL': ['little', 'little']}

response = requests.post('http://127.0.0.1:8080/predict/', data = input)
result = response.json() # 결과를 json 형태로 받아와 저장하지만, python에서는 딕셔너리로 저장됨.
result

{'pred1': [1, 1], 'pred2': ['Leave', 'Leave']}

## 3. 데이터셋(배치)으로 전달하기

* 데이터프레임으로 부터 정보 전달하기1

In [4]:
use_cols = ['INCOME', 'OVERAGE', 'LEFTOVER', 'HOUSE', 'HANDSET_PRICE', 'OVER_15MINS_CALLS_PER_MONTH', 'AVERAGE_CALL_DURATION', 'CHURN', 'REPORTED_SATISFACTION', 'REPORTED_USAGE_LEVEL']
data = pd.read_csv('data/new_data.csv', usecols = use_cols )
target = 'CHURN'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

In [5]:
temp = x.sample(3)

In [6]:
input = temp.to_json(orient = 'columns')
input = json.loads(input)
input

{'INCOME': {'172': 154158.0, '131': 70053.0, '146': 117939.0},
 'OVERAGE': {'172': 0, '131': 0, '146': 269},
 'LEFTOVER': {'172': 9, '131': 41, '146': 16},
 'HOUSE': {'172': None, '131': None, '146': None},
 'HANDSET_PRICE': {'172': 889, '131': 355, '146': 794},
 'OVER_15MINS_CALLS_PER_MONTH': {'172': 1, '131': 1, '146': 15},
 'AVERAGE_CALL_DURATION': {'172': 4, '131': 2, '146': 4},
 'REPORTED_SATISFACTION': {'172': 'very_unsat',
  '131': 'very_unsat',
  '146': 'avg'},
 'REPORTED_USAGE_LEVEL': {'172': 'little',
  '131': 'very_little',
  '146': 'very_little'}}

In [7]:
input = { key : list(temp[key]) for key in list(temp) }
input

{'INCOME': [154158.0, 70053.0, 117939.0],
 'OVERAGE': [0, 0, 269],
 'LEFTOVER': [9, 41, 16],
 'HOUSE': [nan, nan, nan],
 'HANDSET_PRICE': [889, 355, 794],
 'OVER_15MINS_CALLS_PER_MONTH': [1, 1, 15],
 'AVERAGE_CALL_DURATION': [4, 2, 4],
 'REPORTED_SATISFACTION': ['very_unsat', 'very_unsat', 'avg'],
 'REPORTED_USAGE_LEVEL': ['little', 'very_little', 'very_little']}

In [8]:
response = requests.post('http://127.0.0.1:8080/predict/', data = input)
result = response.json() 
result

{'pred1': [0, 0, 1], 'pred2': ['Stay', 'Stay', 'Leave']}

* 데이터프레임으로 부터 정보전달하고, 예측결과 합치기

In [9]:
# request
input = { key : list(x[key]) for key in list(x) }
response = requests.post('http://127.0.0.1:8080/predict/', data = input)
result = response.json() 

In [11]:
# 결과 합치기
x['y'] = y
x['y_pred'] = result['pred1']
x

,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,y,y_pred
0,111527.0,44,76,NaN,751,4,1,very_unsat,little,1,1
1,42472.0,61,0,NaN,156,3,9,very_sat,little,0,0
2,145276.0,66,18,NaN,516,15,4,avg,little,0,1
3,132847.0,0,22,NaN,830,1,5,very_unsat,little,0,0
4,52232.0,138,0,NaN,362,24,14,very_sat,little,1,1
...,...,...,...,...,...,...,...,...,...,...,...
195,31079.0,200,40,NaN,181,20,11,unsat,little,0,1
196,137194.0,63,0,NaN,787,3,11,very_unsat,little,1,0
197,82931.0,0,77,NaN,232,0,2,very_unsat,very_little,0,1
198,124391.0,0,19,NaN,862,0,2,unsat,very_high,0,0


In [12]:
from sklearn.metrics import *

In [13]:
confusion_matrix(x['y'], x['y_pred'])

array([[83, 28],
       [33, 56]], dtype=int64)

In [14]:
print(classification_report(x['y'], x['y_pred']))

              precision    recall  f1-score   support

           0       0.72      0.75      0.73       111
           1       0.67      0.63      0.65        89

    accuracy                           0.69       200
   macro avg       0.69      0.69      0.69       200
weighted avg       0.69      0.69      0.69       200

